# Imports

In [1]:
import plotly.plotly as py
import plotly.graph_objs as go

import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_table_experiments as dt

import numpy as np

import pandas as pd 
# from IPython.display import display
from IPython import display

import json
import os

from parse_input import *

from dill import load
import json

# Load Data

In [2]:
file = 'BSF_year4_v2.xlsx'
my_parser = InputParser(file)

expt, Nqoi = my_parser.parse_expt()
dsgn, Nmp, param_priors = my_parser.parse_dsgn()
sims = my_parser.parse_sim()

input_dict = load(open('linear_surrogate.p', 'rb'))
locals().update(input_dict)

In [3]:
y_samples = y_surrogate(dsgn.values, Θ)

# Static PCP Example

In [4]:
line = dict(color = list(range(len(dsgn))), colorscale = 'Viridis', showscale = True, cmin = 0, cmax = len(dsgn))
dimensions = []
dimensions.append(dict(values = dsgn['ThardB'], label='ThardB'))
dimensions.append(dict(values = dsgn['CO2_PC1'], label='CO2_PC1'))
dimensions.append(dict(values = dsgn['CO2_PC2'], label='CO2_PC2'))
dimensions.append(dict(values = dsgn['O2_PC1'], label='O2_PC1'))
dimensions.append(dict(values = dsgn['O2_PC2'], label='O2_PC2'))
dimensions.append(dict(values = dsgn['AbsCD'], label='AbsCD'))
dimensions.append(dict(values = dsgn['log(Kdeposit)'], label='log(Kdeposit)'))
dimensions.append(dict(values = dsgn['devolTmu'], label='devolTmu'))
dimensions.append(dict(values = dsgn['ln(Avisc)'], label='ln(Avisc)'))
dimensions.append(dict(values = dsgn['Tslag'], label='Tslag'))
dimensions.append(dict(values = dsgn['log(enam)'], label='log(enam)'))
dimensions.append(dict(values = dsgn['Csmag'], label='Csmag'))
dimensions.append(dict(values = dsgn['tSootBlow'], label='tSootBlow'))

par_coords = go.Parcoords(line = line, dimensions = dimensions)

def  test_handler (widget, msg):
    print('Here')
    
par_coords.on_selection(test_handler)
py.iplot([par_coords], filename = 'parcoords')

# Static Error Examples

In [5]:
var = 'O2'
scatter_expt_1 = go.Scatter(name='Experimental', x=expt['m'][var].index, y=expt['m'][var].values, error_y=dict(type='data', array=expt['s'][var].values, visible=True), mode = 'markers')
scatter_sims_1 = go.Scatter(name='Simulation', x=sims.T[var].mean().index, y=sims.T[var].mean(), error_y=dict(type='data', array=sims.T[var].std(), visible=True), mode = 'markers')
scatter_plots = [scatter_expt_1, scatter_sims_1]
py.iplot(scatter_plots, filename = 'scatter')

In [6]:
var = 'HEAT FLUX'
scatter_expt_2 = go.Scatter(name='Experimental', x=expt['m'][var].index, y=expt['m'][var].values, error_y=dict(type='data', array=expt['s'][var].values, visible=True), mode = 'markers')
scatter_sims_2 = go.Scatter(name='Simulation', x=sims.T[var].mean().index, y=sims.T[var].mean(), error_y=dict(type='data', array=sims.T[var].std(), visible=True), mode = 'markers')
scatter_plots = [scatter_expt_2, scatter_sims_2]
py.iplot(scatter_plots, filename = 'scatter')

In [7]:
var = 'TEMPERATURE'
scatter_expt_2 = go.Scatter(name='Experimental', x=expt['m'][var].index, y=expt['m'][var].values, error_y=dict(type='data', array=expt['s'][var].values, visible=True), mode = 'markers')
scatter_sims_2 = go.Scatter(name='Simulation', x=sims.T[var].mean().index, y=sims.T[var].mean(), error_y=dict(type='data', array=sims.T[var].std(), visible=True), mode = 'markers')
scatter_plots = [scatter_expt_2, scatter_sims_2]
py.iplot(scatter_plots)

# Dash Example

In [8]:
dsgn.columns

Index(['ThardB', 'CO2_PC1', 'CO2_PC2', 'O2_PC1', 'O2_PC2', 'AbsCD',
       'log(Kdeposit)', 'devolTmu', 'ln(Avisc)', 'Tslag', 'log(enam)', 'Csmag',
       'tSootBlow'],
      dtype='object')

In [9]:
sims.columns

Index(['Case_1', 'Case_2', 'Case_3', 'Case_4', 'Case_5', 'Case_6', 'Case_7',
       'Case_8', 'Case_9', 'Case_10', 'Case_11', 'Case_12', 'Case_13',
       'Case_14', 'Case_15', 'Base_Case'],
      dtype='object')

In [10]:
input_dict = load(open('linear_surrogate.p', 'rb'))
locals().update(input_dict)
# print(y_surrogate)
# Evaluate the linear surrogate
# y_surrogate = lambda x, Θ: Θ[:, 0] +  x @ Θ[:, 1:].T
y_samples = y_surrogate(dsgn.values, Θ)

In [11]:
y_samples.shape

(16, 170)

In [12]:
sample_data = {}

var_types = set()
for idx in sims.index:
    var_types.add(idx[0])
    
for i, case in enumerate(sims.columns):
    sample_data[case] = {}
    for var in var_types:
        sample_data[case][var] = {}
    for j, idx in enumerate(sims.index):
        var = idx[0]
        loc = idx[1]
        sample_data[case][var][loc] = y_samples[i, j]

In [30]:
output_type='O2'

In [34]:
output_type = output_type.replace('_', ' ')
sample_data = {}

var_types = set()
for idx in sims.index:
    var_types.add(idx[0])

for i, case in enumerate(sims.columns):
    sample_data[case] = {}
    for j, idx in enumerate(sims.index):
        var = idx[0]
        if var == output_type:
            loc = idx[1]
            sample_data[case][loc] = y_samples[i, j]

json.dumps(sample_data)

'{"Case_1": {"Port:3 X:6.3754 Y:0.7747 Z:0.9525": 1.34052760687198, "Port:4 X:6.3754 Y:1.2446 Z:0.9525": 2.372285892252597, "Port:5 X:6.3754 Y:0.1016 Z:1.3335": 0.3778224505442189, "Port:6 X:6.3754 Y:0.3048 Z:1.3335": 0.6640769152187822, "Port:7 X:6.3754 Y:0.7747 Z:1.3335": 0.5861177012072503, "Port:9 X:6.3754 Y:0.1016 Z:2.4765": 0.9268796547241624, "Port:10 X:6.3754 Y:0.3048 Z:2.4765": 1.1548174836529554, "Port:11 X:6.3754 Y:0.7747 Z:2.4765": 3.4097939023774018, "Port:12 X:6.3754 Y:1.2446 Z:2.4765": 2.145144407119716, "Port:13 X:6.3754 Y:0.1016 Z:2.8575": 1.0263972676380373, "Port:14 X:6.3754 Y:0.3048 Z:2.8575": 0.9585495142232139, "Port:15 X:6.3754 Y:0.7747 Z:2.8575": 1.8247181833905337, "Port:16 X:6.3754 Y:1.2446 Z:2.8575": 2.163851690146961, "Port:17 X:9.7155 Y:0.3683 Z:3.2512": 3.308482040521536, "Port:18 X:9.7155 Y:0.3683 Z:2.8702": 3.2041077801810847, "Port:19 X:9.7155 Y:0.3683 Z:2.3876": 3.0104393294543925, "Port:20 X:9.7155 Y:0.3683 Z:1.905": 2.837105473520417, "Port:21 X:9.71

In [35]:
sims.loc[output_type].to_json()

'{"Case_1":{"Port:3 X:6.3754 Y:0.7747 Z:0.9525":1.0670689148,"Port:4 X:6.3754 Y:1.2446 Z:0.9525":1.9100536812,"Port:5 X:6.3754 Y:0.1016 Z:1.3335":0.1705465062,"Port:6 X:6.3754 Y:0.3048 Z:1.3335":0.5166780806,"Port:7 X:6.3754 Y:0.7747 Z:1.3335":0.5590129553,"Port:9 X:6.3754 Y:0.1016 Z:2.4765":0.6848543264,"Port:10 X:6.3754 Y:0.3048 Z:2.4765":0.9297265207,"Port:11 X:6.3754 Y:0.7747 Z:2.4765":3.2633515927,"Port:12 X:6.3754 Y:1.2446 Z:2.4765":2.2343617689,"Port:13 X:6.3754 Y:0.1016 Z:2.8575":0.9254579616,"Port:14 X:6.3754 Y:0.3048 Z:2.8575":0.9365099913,"Port:15 X:6.3754 Y:0.7747 Z:2.8575":1.8975478459,"Port:16 X:6.3754 Y:1.2446 Z:2.8575":2.2458197306,"Port:17 X:9.7155 Y:0.3683 Z:3.2512":3.3031366512,"Port:18 X:9.7155 Y:0.3683 Z:2.8702":3.2484948006,"Port:19 X:9.7155 Y:0.3683 Z:2.3876":3.0724627332,"Port:20 X:9.7155 Y:0.3683 Z:1.905":2.9132673933,"Port:21 X:9.7155 Y:0.3683 Z:1.4224":2.4599806043,"Port:22 X:9.7155 Y:0.3683 Z:0.9398":2.1900366645,"Port:23 X:9.7155 Y:0.3683 Z:0.4572":3.142879